## TOPIC MODELLING (LDA)

In [ ]:
!pip install gensim

In [ ]:
import pandas as pd 
import gensim 
import gensim.corpora as corpora 
from pprint import pprint

In [ ]:
# Load the data from CSV files
preprocessed_reddit_posts = pd.read_csv('preprocessed_reddit_posts.csv') 
preprocessed_twitter_tweets = pd.read_csv('preprocessed_twitter_tweets.csv')

# Concatenate the title and body columns from the reddit DataFrame with the tweet_text column from the twitter DataFrame
merged_df = pd.concat([preprocessed_reddit_posts['title'] + ' ' + preprocessed_reddit_posts['body'], preprocessed_twitter_tweets['tweet_text']])

In [ ]:
# Create a list of documents, where each document is a list of words
documents = [text.split() for text in merged_df]

# Create a dictionary from the documents
vocab = corpora.Dictionary(documents)

# Create a corpus from the documents
corpus = [vocab.doc2bow(text) for text in documents]

# Train an LDA model on the corpus
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=vocab, num_topics=num_topics)

# Print the topics
topic_distribution = pprint(lda_model.print_topics())

In [ ]:
pd.DataFrame(lda_model.get_document_topics(corpus))

In [ ]:
# group the posts by title and count the number of comments per title
grouped = preprocessed_reddit_posts.groupby('title')['num_comments'].sum().reset_index()

# sort the posts by the number of comments in descending order
grouped = grouped.sort_values(by='num_comments', ascending=False)

# print the top 10 posts by the number of comments
print(grouped.head(10))


In [ ]:
topic_distribution_df = pd.DataFrame(topic_distribution, columns=['Topic ' + str(i+1) for i in range(num_topics)])
topic_distribution_df['Dominant Topic'] = topic_distribution_df.idxmax(axis=1)

topic_distribution_table = pd.concat([topic_distribution_df['Dominant Topic'].value_counts(normalize=True),
                                      topic_distribution_df['Dominant Topic'].value_counts()], axis=1)
topic_distribution_table.columns = ['Proportion of Comments', 'Number of Comments']
topic_distribution_table.index.name = 'Topic'
print(topic_distribution_table)